In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
jdk_version = '11'
hadoop_version = 'hadoop2.7'

# Set Environment Variables
os.environ['SPARK_VERSION'] = spark_version
os.environ['JDK_VERSION'] = jdk_version
os.environ['HADOOP_VERSION'] = hadoop_version
os.environ['JAVA_HOME'] = f'/usr/lib/jvm/java-{jdk_version}-openjdk-amd64'
os.environ['SPARK_HOME'] = f'/content/{spark_version}-bin-{hadoop_version}'

# Update modules and install Spark and Java
!apt-get update
!apt-get install openjdk-$JDK_VERSION-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!tar xf $SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!pip install -q findspark

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Level-Two-Starter-Code").getOrCreate()

# Load Amazon Data into Spark DataFrame

In [3]:
from pyspark import SparkFiles
raw_data_file = "amazon_reviews_us_Video_Games_v1_00.tsv.gz"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/" + raw_data_file
spark.sparkContext.addFile(url)
# Read and show the data
df = spark.read.option("sep", "\t").csv("file://" + SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), header=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

# Filter by Votes

In [4]:
# Filter for greater than 20 total votes
df = df.filter('total_votes>20')


In [5]:
# Filter for greater than 50% helpful vote percentage
df= df.filter( (df['helpful_votes']/df['total_votes'] > .5) )

# Describe Stats

In [6]:
# Paid reviews
paid_df = df.filter(df['vine'] == 'Y')
paid_df.describe().show()

+-------+-----------+--------------------+--------------+----------+-------------------+--------------------+----------------+------------------+-----------------+------------------+----+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|product_id|     product_parent|       product_title|product_category|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+----------+-------------------+--------------------+----------------+------------------+-----------------+------------------+----+-----------------+--------------------+--------------------+-----------+
|  count|         90|                  90|            90|        90|                 90|                  90|              90|                90|               90|                90|  90|               90|               

In [7]:
# Unpaid reviews
unpaid_df = df.filter(df['vine'] == 'N')
unpaid_df.describe().show()

+-------+-----------+-------------------+--------------+--------------------+--------------------+--------------------+----------------+------------------+------------------+------------------+-----+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|        customer_id|     review_id|          product_id|      product_parent|       product_title|product_category|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+-------------------+--------------+--------------------+--------------------+--------------------+----------------+------------------+------------------+------------------+-----+-----------------+--------------------+--------------------+-----------+
|  count|      37385|              37385|         37385|               37385|               37385|               37385|           37385|             37385|             37385|          

# Determine the percentage of five-star review among vine reviews

In [15]:
# CODE HERE
paid_number = paid_df.select(['review_id']).count()
paid_five_star_number = paid_df.filter(paid_df['star_rating'] == 5).count()
percentage_five_star_vine = paid_five_star_number/paid_df.select(['star_rating']).count()

print(f'Number of paid reviews {paid_number}')
print(f'Number of paid five star reviews {paid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_vine * 100}%')

Number of paid reviews 90
Number of paid five star reviews 44
Percantage of paid reviews that are five stars 48.888888888888886%


# Determine the percentage of five-star review among non-Vine reviews

In [16]:
unpaid_number = unpaid_df.select(['review_id']).count()
unpaid_five_star_number = unpaid_df.filter(unpaid_df['star_rating'] == 5).count()
percentage_five_star_non_vine = unpaid_five_star_number/unpaid_df.select(['star_rating']).count()


print(f'Number of unpaid reviews {unpaid_number}')
print(f'Number of unpaid five star reviews {unpaid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_non_vine * 100}%')

Number of unpaid reviews 37385
Number of unpaid five star reviews 14626
Percantage of paid reviews that are five stars 39.122642771164905%


# Findings

From the analysis, we can conclude that vine reviews are on average rated higher and with more 5 star reviews then non-vine reviews. There were also more votes per review and were found more helpful. 